In [16]:
!pip install wget

In [2]:
import wget
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train (1).txt'

In [17]:
import pandas as pd

In [18]:
df_review = pd.read_csv('ratings_train.txt', sep='\t')
df_review[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [5]:
!pip install python-mecab-ko

In [6]:
from mecab import MeCab
mecab = MeCab()

In [55]:
df_review.dropna(subset=['document', 'label'], inplace=True)
df_review.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [8]:
type(df_review), type(df_review['document'])

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [9]:
import re
document_clear=[]
# 마침표, 말줄임표, 따옴표, 쌍따옴표 등의 문장부호들 삭제하기
for i in df_review['document']:
    i = i.lower()
    i = re.sub('[\.,!?\"\']', '',i) 
    document_clear.append(i)
document_clear[1]

'흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나'

In [10]:
# 형태소 단위로 분리
!pip install python-mecab-ko

In [11]:
from mecab import MeCab
mecab = MeCab()

In [26]:
mecab.pos(document_clear[1]), mecab.pos(document_clear[2])

([('흠', 'IC'),
  ('포스터', 'NNP'),
  ('보고', 'JKB'),
  ('초딩', 'NNG'),
  ('영화', 'NNG'),
  ('줄', 'NNG'),
  ('오버', 'NNG'),
  ('연기', 'NNG'),
  ('조차', 'JX'),
  ('가볍', 'VA'),
  ('지', 'EC'),
  ('않', 'VX'),
  ('구나', 'EC')],
 [('너무', 'MAG'), ('재', 'XPN'), ('밓었다그래서보는것을추천한다', 'UNKNOWN')])

In [50]:
len(document_clear)

149995

In [47]:
# NNP, JKB, MAG, IC, NNG 를 제외한 것들 삭제
document_token = []
for item in document_clear :
    token_list = mecab.pos(item)
    token = ' '
    for i in token_list :
        if i[1] == 'NNP' or i[1] == 'JKB' or i[1] == 'MAG' or i[1] == 'IC' or i[1] == 'NNG' or i[1]=='VA' or i[1]=='JX':
            token = token + ' ' + i[0]
    document_token.append(token)

In [48]:
document_token[:3], len(document_token)

(['  아 더 빙 진짜 짜증 목소리', '  흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍', '  너무'], 149995)

In [58]:
df_review['document_clear'] = document_token
df_review[:3]

,id,document,label,document_clear
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더 빙 진짜 짜증 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무


In [77]:
# 백터화 진행
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=None)
tfidf.fit(df_review['document_clear'])

TfidfVectorizer()

In [67]:
#train_set과 test_set분리
from sklearn.model_selection import train_test_split

In [72]:
feature = df_review['document_clear']
target = df_review['label']
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.3, random_state=2)
feature_train.shape,feature_test.shape, target_train.shape, target_test.shape

((104996,), (44999,), (104996,), (44999,))

In [78]:
# 모델 사용
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [83]:
feature_train_vector = tfidf.transform(feature_train)
model.fit(feature_train_vector, target_train)

DecisionTreeClassifier()

## 평가

In [86]:
feature_test_vector = tfidf.transform(feature_test)

In [87]:
target_test_predict = model.predict(feature_test_vector)

In [89]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.70      0.75      0.72     22481
           1       0.73      0.67      0.70     22518

    accuracy                           0.71     44999
   macro avg       0.71      0.71      0.71     44999
weighted avg       0.71      0.71      0.71     44999



### 결론 : f1이 높으며 그룹간의 f1-score의 차이가 0.02로 근소하기 때문에 분류 신뢰성이 좋음
### 모델 성능은 70%